# 02 — Data Preprocessing
### CSC 577 | Team Rocket | Food.com Recipe Recommender

This notebook cleans and filters the raw data based on decisions made in `01_EDA.ipynb`.

**Steps:**
1. Load raw data
2. Clean interactions (remove zero ratings, duplicates)
3. Filter to active users (≥ 10 ratings) and popular recipes (≥ 5 ratings)
4. Clean and parse recipe metadata (ingredients, tags, nutrition)
5. Merge and align the two datasets
6. Save processed files to `data/processed/`

## 0. Imports

In [1]:
import pandas as pd
import numpy as np
import ast
import os

# Create output directory if it doesn't exist
os.makedirs('../data/processed', exist_ok=True)

print('Libraries loaded.')

Libraries loaded.


## 1. Load Raw Data

In [2]:
recipes_raw = pd.read_csv('../data/RAW_recipes.csv')
interactions_raw = pd.read_csv('../data/RAW_interactions.csv')

print(f'Raw recipes shape:      {recipes_raw.shape}')
print(f'Raw interactions shape: {interactions_raw.shape}')

Raw recipes shape:      (231637, 12)
Raw interactions shape: (1132367, 5)


## 2. Clean Interactions

### 2a. Remove zero ratings
A rating of `0` on Food.com means the user left a text review but did NOT give a star rating. These are not valid for our recommender.

In [3]:
interactions = interactions_raw[interactions_raw['rating'] > 0].copy()

removed_zeros = len(interactions_raw) - len(interactions)
print(f'Removed {removed_zeros:,} zero-rating rows')
print(f'Interactions remaining: {len(interactions):,}')

Removed 60,847 zero-rating rows
Interactions remaining: 1,071,520


### 2b. Remove duplicate interactions
Keep the most recent rating if a user rated the same recipe more than once.

In [4]:
# Sort by date descending so keep='first' keeps the most recent
interactions['date'] = pd.to_datetime(interactions['date'])
interactions = interactions.sort_values('date', ascending=False)
interactions = interactions.drop_duplicates(subset=['user_id', 'recipe_id'], keep='first')

print(f'After removing duplicates: {len(interactions):,} interactions')

After removing duplicates: 1,071,520 interactions


## 3. Filter Active Users and Popular Recipes

We apply iterative filtering — removing sparse users and recipes in turns — until the dataset stabilizes. This is standard practice to ensure the CF model has enough signal.

In [5]:
MIN_USER_RATINGS   = 10   # minimum ratings a user must have
MIN_RECIPE_RATINGS = 5    # minimum ratings a recipe must have

def filter_interactions(df, min_user, min_recipe):
    """Iteratively filter until no users or recipes fall below the threshold."""
    iteration = 0
    while True:
        iteration += 1
        start_size = len(df)

        # Filter users
        user_counts = df['user_id'].value_counts()
        valid_users = user_counts[user_counts >= min_user].index
        df = df[df['user_id'].isin(valid_users)]

        # Filter recipes
        recipe_counts = df['recipe_id'].value_counts()
        valid_recipes = recipe_counts[recipe_counts >= min_recipe].index
        df = df[df['recipe_id'].isin(valid_recipes)]

        end_size = len(df)
        print(f'  Iteration {iteration}: {start_size:,} → {end_size:,} interactions')

        if start_size == end_size:
            print('  Converged.')
            break

    return df

print('Applying iterative filtering...')
interactions_filtered = filter_interactions(interactions.copy(), MIN_USER_RATINGS, MIN_RECIPE_RATINGS)

print(f'\nFinal interactions: {len(interactions_filtered):,}')
print(f'Unique users:       {interactions_filtered["user_id"].nunique():,}')
print(f'Unique recipes:     {interactions_filtered["recipe_id"].nunique():,}')

Applying iterative filtering...
  Iteration 1: 1,071,520 → 498,242 interactions
  Iteration 2: 498,242 → 472,335 interactions
  Iteration 3: 472,335 → 471,334 interactions
  Iteration 4: 471,334 → 471,303 interactions
  Iteration 5: 471,303 → 471,303 interactions
  Converged.

Final interactions: 471,303
Unique users:       9,087
Unique recipes:     36,916


In [6]:
# Verify thresholds were met
user_rating_counts   = interactions_filtered['user_id'].value_counts()
recipe_rating_counts = interactions_filtered['recipe_id'].value_counts()

assert user_rating_counts.min() >= MIN_USER_RATINGS, 'Some users below threshold!'
assert recipe_rating_counts.min() >= MIN_RECIPE_RATINGS, 'Some recipes below threshold!'

print(f'Min ratings per user:   {user_rating_counts.min()}')
print(f'Max ratings per user:   {user_rating_counts.max()}')
print(f'Min ratings per recipe: {recipe_rating_counts.min()}')
print(f'Max ratings per recipe: {recipe_rating_counts.max()}')
print('\nAll threshold checks passed.')

Min ratings per user:   10
Max ratings per user:   2712
Min ratings per recipe: 5
Max ratings per recipe: 780

All threshold checks passed.


## 4. Clean Recipe Metadata

### 4a. Keep only recipes that appear in filtered interactions

In [7]:
valid_recipe_ids = interactions_filtered['recipe_id'].unique()
recipes = recipes_raw[recipes_raw['id'].isin(valid_recipe_ids)].copy()

print(f'Recipes kept: {len(recipes):,} out of {len(recipes_raw):,}')

Recipes kept: 36,916 out of 231,637


### 4b. Parse stringified list columns

In [8]:
def safe_parse_list(x):
    """Safely parse a stringified Python list."""
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return []

recipes['ingredients'] = recipes['ingredients'].apply(safe_parse_list)
recipes['tags']        = recipes['tags'].apply(safe_parse_list)
recipes['steps']       = recipes['steps'].apply(safe_parse_list)

print('Parsed: ingredients, tags, steps')
print(f'Example ingredients for first recipe:')
print(recipes['ingredients'].iloc[0])

Parsed: ingredients, tags, steps
Example ingredients for first recipe:
['lean pork chops', 'flour', 'salt', 'dry mustard', 'garlic powder', 'oil', 'chicken rice soup']


### 4c. Parse nutrition column

Nutrition is stored as `[calories, total_fat_%DV, sugar_%DV, sodium_%DV, protein_%DV, sat_fat_%DV, carbs_%DV]`

In [9]:
nutrition_cols = ['calories', 'total_fat_pdv', 'sugar_pdv', 'sodium_pdv',
                  'protein_pdv', 'sat_fat_pdv', 'carbs_pdv']

nutrition_parsed = recipes['nutrition'].apply(safe_parse_list)
recipes[nutrition_cols] = pd.DataFrame(nutrition_parsed.tolist(), index=recipes.index)

print('Nutrition columns added:')
print(recipes[nutrition_cols].describe().round(2))

Nutrition columns added:
       calories  total_fat_pdv  sugar_pdv  sodium_pdv  protein_pdv  \
count  36916.00       36916.00   36916.00    36916.00     36916.00   
mean     431.51          32.51      70.65       29.64        33.49   
std      665.59          64.74     202.72      106.20        64.57   
min        0.00           0.00       0.00        0.00         0.00   
25%      167.50           8.00       8.00        5.00         6.00   
50%      294.80          19.00      22.00       15.00        17.00   
75%      483.62          38.00      60.00       33.00        50.00   
max    38680.10        4331.00    8320.00     7084.00      6552.00   

       sat_fat_pdv  carbs_pdv  
count     36916.00   36916.00  
mean         40.99      13.93  
std          84.91      26.31  
min           0.00       0.00  
25%           7.00       3.00  
50%          21.00       8.00  
75%          49.00      15.00  
max        4969.00    1188.00  


### 4d. Handle outliers in minutes and nutrition

In [10]:
# Remove recipes with 0 or unreasonably high cook times (> 1 week = 10080 min)
before = len(recipes)
recipes = recipes[(recipes['minutes'] > 0) & (recipes['minutes'] <= 10080)]
print(f'Removed {before - len(recipes)} recipes with invalid cook times')

# Cap extreme calorie values at 99th percentile
cal_99 = recipes['calories'].quantile(0.99)
recipes['calories'] = recipes['calories'].clip(upper=cal_99)
print(f'Calories capped at 99th percentile: {cal_99:.1f}')

Removed 160 recipes with invalid cook times
Calories capped at 99th percentile: 2967.5


### 4e. Drop unused columns and handle missing values

In [11]:
# Drop raw nutrition string (already parsed) and steps (not needed for CF)
recipes = recipes.drop(columns=['nutrition', 'steps'])

# Fill missing descriptions with empty string
recipes['description'] = recipes['description'].fillna('')

# Rename 'id' to 'recipe_id' for consistency with interactions
recipes = recipes.rename(columns={'id': 'recipe_id'})

print('Remaining recipe columns:')
print(recipes.columns.tolist())
print(f'\nMissing values:')
print(recipes.isnull().sum())

Remaining recipe columns:
['name', 'recipe_id', 'minutes', 'contributor_id', 'submitted', 'tags', 'n_steps', 'description', 'ingredients', 'n_ingredients', 'calories', 'total_fat_pdv', 'sugar_pdv', 'sodium_pdv', 'protein_pdv', 'sat_fat_pdv', 'carbs_pdv']

Missing values:
name              0
recipe_id         0
minutes           0
contributor_id    0
submitted         0
tags              0
n_steps           0
description       0
ingredients       0
n_ingredients     0
calories          0
total_fat_pdv     0
sugar_pdv         0
sodium_pdv        0
protein_pdv       0
sat_fat_pdv       0
carbs_pdv         0
dtype: int64


## 5. Final Alignment Check

Make sure every recipe_id in interactions has a corresponding row in recipes (and vice versa).

In [12]:
interaction_recipe_ids = set(interactions_filtered['recipe_id'].unique())
recipe_ids             = set(recipes['recipe_id'].unique())

in_interactions_not_recipes = interaction_recipe_ids - recipe_ids
in_recipes_not_interactions = recipe_ids - interaction_recipe_ids

print(f'Recipe IDs in interactions but not in recipes: {len(in_interactions_not_recipes)}')
print(f'Recipe IDs in recipes but not in interactions: {len(in_recipes_not_interactions)}')

# If any mismatch, remove those interactions
if in_interactions_not_recipes:
    print('Removing mismatched interactions...')
    interactions_filtered = interactions_filtered[
        interactions_filtered['recipe_id'].isin(recipe_ids)
    ]
    print(f'Interactions after alignment: {len(interactions_filtered):,}')

print('\nAlignment OK.')

Recipe IDs in interactions but not in recipes: 160
Recipe IDs in recipes but not in interactions: 0
Removing mismatched interactions...
Interactions after alignment: 469,637

Alignment OK.


## 6. Create User and Item ID Mappings

Many CF libraries (including Surprise) work better with contiguous integer IDs. We create mappings here and store them for later use.

In [13]:
# Map original IDs to 0-indexed integers
unique_users   = sorted(interactions_filtered['user_id'].unique())
unique_recipes = sorted(interactions_filtered['recipe_id'].unique())

user2idx   = {uid: idx for idx, uid in enumerate(unique_users)}
recipe2idx = {rid: idx for idx, rid in enumerate(unique_recipes)}
idx2user   = {v: k for k, v in user2idx.items()}
idx2recipe = {v: k for k, v in recipe2idx.items()}

interactions_filtered['user_idx']   = interactions_filtered['user_id'].map(user2idx)
interactions_filtered['recipe_idx'] = interactions_filtered['recipe_id'].map(recipe2idx)

print(f'Number of users:   {len(unique_users):,}')
print(f'Number of recipes: {len(unique_recipes):,}')
print('\nSample mapping (user_id → user_idx):')
print(dict(list(user2idx.items())[:5]))

Number of users:   9,087
Number of recipes: 36,756

Sample mapping (user_id → user_idx):
{np.int64(1533): 0, np.int64(1535): 1, np.int64(1634): 2, np.int64(1676): 3, np.int64(1792): 4}


## 7. Final Sparsity Check

In [14]:
n_users   = len(unique_users)
n_recipes = len(unique_recipes)
n_ratings = len(interactions_filtered)
sparsity  = 1 - (n_ratings / (n_users * n_recipes))

print('=== Final Processed Dataset Stats ===')
print(f'  Users:       {n_users:,}')
print(f'  Recipes:     {n_recipes:,}')
print(f'  Ratings:     {n_ratings:,}')
print(f'  Sparsity:    {sparsity*100:.4f}%')
print(f'  Rating mean: {interactions_filtered["rating"].mean():.2f}')
print(f'  Rating std:  {interactions_filtered["rating"].std():.2f}')

=== Final Processed Dataset Stats ===
  Users:       9,087
  Recipes:     36,756
  Ratings:     469,637
  Sparsity:    99.8594%
  Rating mean: 4.73
  Rating std:  0.59


## 8. Save Processed Data

In [15]:
# Save interactions (only keep columns we need)
interactions_out = interactions_filtered[['user_id', 'recipe_id', 'rating', 'date', 'user_idx', 'recipe_idx']]
interactions_out.to_csv('../data/processed/interactions_clean.csv', index=False)
print(f'Saved interactions_clean.csv — {len(interactions_out):,} rows')

# Save recipes — store ingredients and tags as pipe-separated strings
# (easier to reload than stringified lists)
recipes_out = recipes.copy()
recipes_out['ingredients'] = recipes_out['ingredients'].apply(lambda x: '|'.join(x))
recipes_out['tags']        = recipes_out['tags'].apply(lambda x: '|'.join(x))
recipes_out.to_csv('../data/processed/recipes_clean.csv', index=False)
print(f'Saved recipes_clean.csv      — {len(recipes_out):,} rows')

# Save ID mappings
pd.DataFrame(list(user2idx.items()), columns=['user_id', 'user_idx']).to_csv(
    '../data/processed/user_mapping.csv', index=False)
pd.DataFrame(list(recipe2idx.items()), columns=['recipe_id', 'recipe_idx']).to_csv(
    '../data/processed/recipe_mapping.csv', index=False)
print('Saved user_mapping.csv and recipe_mapping.csv')

Saved interactions_clean.csv — 469,637 rows
Saved recipes_clean.csv      — 36,756 rows
Saved user_mapping.csv and recipe_mapping.csv


## 9. Quick Sanity Check on Saved Files

In [16]:
# Reload and verify
check_interactions = pd.read_csv('../data/processed/interactions_clean.csv')
check_recipes      = pd.read_csv('../data/processed/recipes_clean.csv')

print('=== Reloaded interactions_clean.csv ===')
print(check_interactions.shape)
print(check_interactions.head(3))

print('\n=== Reloaded recipes_clean.csv ===')
print(check_recipes.shape)
print(check_recipes[['recipe_id', 'name', 'ingredients', 'tags']].head(3))

=== Reloaded interactions_clean.csv ===
(469637, 6)
      user_id  recipe_id  rating        date  user_idx  recipe_idx
0      226867     363072       5  2018-12-18      3472       33295
1     1290903     131607       5  2018-12-18      8508       17700
2  2001513060     192495       5  2018-12-17      9082       23370

=== Reloaded recipes_clean.csv ===
(36756, 17)
   recipe_id                              name  \
0      63986  chicken lickin  good  pork chops   
1      43026                    chile rellenos   
2      23933                    chinese  candy   

                                         ingredients  \
0  lean pork chops|flour|salt|dry mustard|garlic ...   
1  egg roll wrap|whole green chilies|cheese|corns...   
2  butterscotch chips|chinese noodles|salted peanuts   

                                                tags  
0  weeknight|time-to-make|course|main-ingredient|...  
1  60-minutes-or-less|time-to-make|course|main-in...  
2  15-minutes-or-less|time-to-make|course

## Summary

| Step | Action |
|---|---|
| Remove zero ratings | Dropped interactions with `rating == 0` |
| Remove duplicates | Kept most recent rating per user-recipe pair |
| Filter users | Kept users with ≥ 10 ratings (iterative) |
| Filter recipes | Kept recipes with ≥ 5 ratings (iterative) |
| Parse ingredients/tags | `ast.literal_eval` → pipe-separated strings |
| Parse nutrition | Expanded into 7 named columns |
| Outlier handling | Removed 0-minute and >10080-minute recipes, capped calories |
| ID mappings | Created `user_idx` and `recipe_idx` for CF model |